In [ ]:
from __future__ import division, print_function

import safe_rl
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### Test the triangulation

Create a triangulation on a grid and plot the corresponding results.
Then take a test point and return in which simplex the corresponding point lies

In [ ]:
limits = [[-1, 1], [-1, 1]]
npoints = [2, 3]

# Create triangulation
tri = safe_rl.Delaunay(limits, npoints)

In [ ]:
# Get all simplices and points
simplices = tri.simplices(np.arange(tri.nsimplex))
points = tri.index_to_state(np.arange(tri.nindex))

# Plot nodes
plt.plot(points[:, 0], points[:, 1], 'o')

# Annotate nodes
for p in points:
    plt.text(p[0] - 0.03, p[1] + 0.03,
             tri.state_to_index(p)[0], ha='right')

# Plot triangles
plt.triplot(points[:,0], points[:,1], simplices.copy())

# Annotate triangles
for i, simplex in enumerate(simplices):
    p = points[simplex].mean(axis=0)
    plt.text(p[0], p[1],
             '#%d' % i, ha='center')

# Format plot nicely
offset = 0.1 * np.array([-1, 1])
plt.xlim(tri.limits[0] + offset * np.diff(tri.limits[0]))
plt.ylim(tri.limits[1] + offset * np.diff(tri.limits[1]))


# test points
test_point = np.array([[0.9, 0.9],
                       [-0.5, -0.2]])
plt.plot(test_point[0], test_point[1], 'x')
plt.show()

# Output the result of the evaluation
simplices = tri.find_simplex(test_point)
simplex_nodes = tri.simplices(simplices)

print('Evaluation results for test points:\n')
for index, nodes in zip(simplices, simplex_nodes):
    print("simplex {id} with nodes {nodes}".format(id=index, nodes=nodes))
